In [2]:
import pandas as pd

In [8]:
df = pd.read_excel("robery_rate_df.xlsx")

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [ ]:
df.head()

,Marca,Quantidade de Roubos
0,FORD/KA SEL 1.5 SD,160
1,GM/CLASSIC LIFE,1955
2,TOYOTA/COROLLA XEI20FLEX,3181
3,I/VW JETTA 2.0T,688
4,I/AUDI A4 2.0TFSI,128


In [ ]:
# Remove espaços extras
df["Marca"] = df["Marca"].str.strip()

# Separa marca e modelo tratando ambos os formatos
def separar_marca_modelo(valor):
    if pd.isna(valor):
        return None, None

    valor = valor.strip()

    # Caso 1: tem "I/" no começo (ex: I/VW JETTA)
    if valor.startswith("I/"):
        partes = valor.split(" ", 1)
        if len(partes) > 1:
            marca_parte = partes[0].replace("I/", "").strip()  # tira o "I/"
            modelo_parte = partes[1].strip()
            return marca_parte, modelo_parte

    # Caso 2: formato "VW/JETTA"
    elif "/" in valor:
        partes = valor.split("/")
        if len(partes) == 2:
            return partes[0].strip(), partes[1].strip(" ")

    # Caso 3: formato inesperado → tenta separar por espaço
    partes = valor.split(" ", 1)
    if len(partes) == 2:
        return partes[0].strip(), partes[1].strip()
    else:
        return valor, None


# Aplica a função à coluna
df[["Marca", "Modelo"]] = df["Marca"].apply(
    lambda x: pd.Series(separar_marca_modelo(x))
)


In [ ]:
df.head(5)

,Marca,Quantidade de Roubos,Modelo
0,FORD,160,KA SEL 1.5 SD
1,GM,1955,CLASSIC LIFE
2,TOYOTA,3181,COROLLA XEI20FLEX
3,VW,688,JETTA 2.0T
4,AUDI,128,A4 2.0TFSI


In [ ]:
# Formata o modelo
df = df.dropna()
df["Modelo"] = df["Modelo"].str.split(" ").str[0]

In [ ]:
df.size

31023

In [ ]:
# Agrupo modelos

df_grouped = (
    df.groupby(["Modelo", "Marca"], as_index=False)["Quantidade de Roubos"]
      .sum()
)# df_grouped.size
df_grouped.head()
print(df_grouped.size)

3762


In [ ]:
df.head()

,Marca,Quantidade de Roubos,Modelo
0,FORD,160,KA
1,GM,1955,CLASSIC
2,TOYOTA,3181,COROLLA
3,VW,688,JETTA
4,AUDI,128,A4


In [ ]:
df_clean = df_grouped.dropna()
df_clean = df_clean[(df_clean["Modelo"] != "") & (df_clean["Marca"] != "")]
df_clean

,Modelo,Marca,Quantidade de Roubos
0,03,CODIGO NULO - 01,1
1,10.160,VW,4
2,106,PEUGEOT,7
3,107,PEUGEOT,1
4,11.180,VW,3
...,...,...,...
1249,ZAFIRA,GM,2127
1250,ZHONGYI,HAFEI,1
1251,ZILK,VW,13
1252,ZR-V,HONDA,4


In [ ]:
df_clean[df_clean["Modelo"].str.contains("JETTA")]

,Modelo,Marca,Quantidade de Roubos
703,JETTA,VOLKSWAGEN,45
704,JETTA,VW,3725


In [ ]:
classes = [
    {
        "name": "VOLKSWAGEN",
        "possibilities": ["VOLKSWAGEN", "VW"]
    },
    {
        "name": "CHEVROLET",
        "possibilities": ["CHEVROLET", "GM", "CHEV"]
    },
    {
        "name": "LAND ROVER",
        "possibilities": ["LR", "LAND ROVER"]
    },
    {
        "name": "FORD",
        "possibilities": ["FORD"]
    },
    {
        "name": "HYUNDAI",
        "possibilities": ["HYUNDAI"]
    },
    {
        "name": "HONDA",
        "possibilities": ["HONDA"]
    },
    {
        "name": "RENAULT",
        "possibilities": ["RENAULT"]
    },
    {
        "name": "PORSCHE",
        "possibilities": ["PORSCHE"]
    },
    {
        "name": "PEUGEOT",
        "possibilities": ["PEUGEOT"]
    },
    {
        "name": "JEEP",
        "possibilities": ["JEEP"]
    },
]

In [ ]:
# Criar dicionário: cada possibilidade mapeia para o nome principal
marca_map = {}
for classe in classes:
    for poss in classe["possibilities"]:
        marca_map[poss.upper()] = classe["name"].upper()


In [ ]:
marca_map

{'VOLKSWAGEN': 'VOLKSWAGEN',
 'VW': 'VOLKSWAGEN',
 'CHEVROLET': 'CHEVROLET',
 'GM': 'CHEVROLET',
 'CHEV': 'CHEVROLET',
 'LR': 'LAND ROVER',
 'LAND ROVER': 'LAND ROVER',
 'FORD': 'FORD',
 'HYUNDAI': 'HYUNDAI',
 'HONDA': 'HONDA',
 'RENAULT': 'RENAULT',
 'PORSCHE': 'PORSCHE',
 'PEUGEOT': 'PEUGEOT',
 'JEEP': 'JEEP'}

In [ ]:
df_clean["Marca"] = df_clean["Marca"].str.upper().map(marca_map)
df_clean = df_clean.groupby(["Marca", "Modelo"], as_index=False)["Quantidade de Roubos"].sum()


In [1]:
# df_clean[df_clean["Modelo"].str.contains("POLO")]
df_clean[df_clean["Modelo"] == "Dolphin"]

NameError: name 'df_clean' is not defined